In [14]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

root = Path("/mnt/ssd-1/alexm/w2s/results/")

run_name = "sft_2000"
run_dir = root / run_name

df = pd.read_json(run_dir / "results.json", orient="records", lines=True)    

In [10]:
df

,ids,num_oracle,oracle_ids,num_weak,calibrated_logodds,gt_soft_labels,auroc,accuracy,weak_auc,weak_acc,weak_soft_labels
0,"[43cab065, 9f7f2cac, 7d4a98f3, a5647cc1, b2f2b...",0,[],2500,"[0.7265625, 1.3359375, 0.58203125, 0.416015625...","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.780532,0.6745,0.781921,0.6620,"[0.8158201575, 0.8152325153000001, 0.184179842..."
1,"[43cab065, 9f7f2cac, 7d4a98f3, a5647cc1, b2f2b...",1,[8871a677],2500,"[0.1181640625, 0.205078125, 0.12890625, 0.1206...","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.674900,0.5080,0.680893,0.4875,"[0.8158201575, 0.8152325153000001, 0.184179842..."
2,"[43cab065, 9f7f2cac, 7d4a98f3, a5647cc1, b2f2b...",2,"[0040da5b, 8871a677]",2500,"[0.0185546875, 0.0971679688, -0.0368652344, -0...","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.791695,0.6340,0.787998,0.6525,"[0.8158201575, 0.8152325153000001, 0.184179842..."
3,"[43cab065, 9f7f2cac, 7d4a98f3, a5647cc1, b2f2b...",4,"[b947bba2, 0040da5b, 8871a677, 333942b7]",2500,"[0.0258789062, 0.1044921875, -0.0356445312, -0...","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.792623,0.6415,0.787778,0.6580,"[0.8158201575, 0.8152325153000001, 0.184179842..."
4,"[43cab065, 9f7f2cac, 7d4a98f3, a5647cc1, b2f2b...",8,"[333942b7, 81adc53e, d646de83, d60d07ec, 0040d...",2500,"[-0.021484375, 0.1015625, -0.0344238281, -0.09...","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.793210,0.6325,0.795289,0.6520,"[0.8158201575, 0.8152325153000001, 0.184179842..."
5,"[43cab065, 9f7f2cac, 7d4a98f3, a5647cc1, b2f2b...",16,"[333942b7, bc9c2f7f, a9e8fbe3, 81adc53e, d646d...",2500,"[-0.005859375, 0.0668945312, -0.04296875, -0.1...","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.794033,0.6170,0.792616,0.6355,"[0.8158201575, 0.8152325153000001, 0.184179842..."
6,"[43cab065, 9f7f2cac, 7d4a98f3, a5647cc1, b2f2b...",32,"[fb5b109c, 005792cf, 0040da5b, 8871a677, 98e35...",2500,"[0.0197753906, 0.0849609375, -0.037109375, -0....","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.791786,0.6355,0.789467,0.6540,"[0.8158201575, 0.8152325153000001, 0.184179842..."
7,"[43cab065, 9f7f2cac, 7d4a98f3, a5647cc1, b2f2b...",64,"[71d184c1, fb5b109c, 005792cf, 789518e5, 37585...",2500,"[0.047119140600000005, 0.125, -0.0292968750000...","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.793332,0.6515,0.788636,0.6660,"[0.8158201575, 0.8152325153000001, 0.184179842..."
8,"[43cab065, 9f7f2cac, 7d4a98f3, a5647cc1, b2f2b...",128,"[71d184c1, aed8e814, 2e0daec3, 96ca08fe, 7107b...",2500,"[0.1220703125, 0.283203125, 0.0085449219, -0.0...","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.788415,0.6905,0.784169,0.6940,"[0.8158201575, 0.8152325153000001, 0.184179842..."
9,"[43cab065, 9f7f2cac, 7d4a98f3, a5647cc1, b2f2b...",256,"[71d184c1, cfe0f53f, 3b959758, aed8e814, 02695...",2500,"[0.271484375, 0.625, 0.0830078125, -0.01501464...","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.794451,0.6730,0.791871,0.6695,"[0.8158201575, 0.8152325153000001, 0.184179842..."
